In [7]:
from inference_sdk import InferenceHTTPClient

import cv2
from PIL import Image
import pytesseract
import fitz
import spacy

In [3]:
def pdf_to_jpg(pdf_path):
    pdf_document = fitz.open(pdf_path)
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        output_path = f"cv_{page_number + 1}.jpg"
        img.save(output_path, "JPEG")
        return output_path


In [12]:
def get_skills(text):
    nlp=spacy.load("en_core_web_lg",disable=['ner'])
    ruler = nlp.add_pipe("entity_ruler")
    ruler.from_disk('jz_skill_patterns.jsonl')
    doc = nlp(text.lower())
    list_skills = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            list_skills.append(ent.text.lower())  # lower
    list_skills = list(set(list_skills))
    return ' '.join(list_skills)

In [13]:
def get_skills_experience_and_project_cv(path):
    image_path = pdf_to_jpg(path)
    CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="BIrE2FC1DoCP2tdXB9MA"
    )
    
    # Define the model ID
    model_id = "resume-parser-bchlq/1"
    result = CLIENT.infer(image_path, model_id=model_id)
    
    image = cv2.imread(image_path)

    skills_text = ''
    for prediction in result['predictions']:
        if prediction['class'] == 'Experience':
            x = int(prediction['x'] - prediction['width'] / 2)
            y = int(prediction['y'] - prediction['height'] / 2)
            width = int(prediction['width'])
            height = int(prediction['height'])
            
            skills_image = image[y:y+height, x:x+width]            
            skills_image_rgb = cv2.cvtColor(skills_image, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(skills_image_rgb)
            skills_text += pytesseract.image_to_string(pil_image)
        
        if prediction['class'] == 'Projects':
            x = int(prediction['x'] - prediction['width'] / 2)
            y = int(prediction['y'] - prediction['height'] / 2)
            width = int(prediction['width'])
            height = int(prediction['height'])
            
            skills_image = image[y:y+height, x:x+width]            
            skills_image_rgb = cv2.cvtColor(skills_image, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(skills_image_rgb)
            skills_text += pytesseract.image_to_string(pil_image)
            
    return get_skills(skills_text)

In [15]:
get_skills_experience_and_project_cv('elbahri chayma.pdf')

'image processing unity ml web mining raspberry pi machine learning algorithm opencv python artificial intelligence database arduino software nlp algorithms robot testing simulation computer vision angular mysql pandas tensorflow firebase selenium deployment numpy'